In [ ]:
transform = transforms.Compose([
    # Randomly flip images horizontally
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    # Convert images to PyTorch tensors and scale to [0, 1]
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])



animal_dataset = datasets.ImageFolder(root='/kaggle/input/animals/animals',  # Specify the root directory of the dataset
                               transform=transform)  # Apply the defined transformations to the dataset


dataloader = torch.utils.data.DataLoader(animal_dataset, 32, shuffle=True, num_workers=2)

In [ ]:
class GetPatchEmbeddings(nn.Module):
    def __init__(self, patch_size=16):
        super(GetPatchEmbeddings, self).__init__()
        self.patch_size = patch_size

    def divide_into_patches(self,image , patch_size=16):
        """
        Divides an image into non-overlapping patches.

        Args:
            image (torch.Tensor): Input image of shape (C, H, W).
            patch_size (int): Size of each square patch (patch_size x patch_size).

        Returns:
            torch.Tensor: A tensor of patches of shape (num_patches, C, patch_size, patch_size).
        """
        image = image.to(device)
        B,C, H, W = image.shape
        NUM_PATCHES = int(H*W/(patch_size**2))
        patches = image.unfold(2, patch_size, patch_size).unfold(3, patch_size, patch_size).to(device)
        patches = patches.permute(0,2, 3, 1, 4, 5).reshape(B, NUM_PATCHES,C, patch_size, patch_size).to(device)
        return patches

    def sample_context_blocks(self,patches, m=49, image_size=(128,128), patch_size=16 ):
        """
        Randomly selects m continuous patches forming a square block (LxL).

        Args:
            patches (torch.Tensor): Tensor of patches of shape (num_patches, C, patch_size, patch_size).
            image_size (tuple): Original image size as (H, W).
            patch_size (int): Size of each patch (patch_size x patch_size).
            m (int): Number of continuous patches to select.

        Returns:
            list: Indices of selected patches.
            torch.Tensor: Selected patches of shape (m, C, patch_size, patch_size).
        """
        H, W = image_size
        n_patches_h = H // patch_size
        n_patches_w = W // patch_size
        patches = patches.to(device)
        # Compute side length of the square block (LxL = m)
        side_length = int(m ** 0.5)
        assert side_length ** 2 == m, "m must be a perfect square to form a square block."

        # Randomly select a top-left corner for the square block
        max_row = n_patches_h - side_length
        max_col = n_patches_w - side_length
        start_row = random.randint(0, max_row)
        start_col = random.randint(0, max_col)

        # Collect indices of the patches in the square block
        context_indices = []
        for _ in range(patches.shape[0]):
            context_indices.append([
            (start_row + i) * n_patches_w + (start_col + j)
            for i in range(side_length)
            for j in range(side_length)
        ])

        masked_patches = patches.clone().to(device)
        for j in range(patches.shape[0]):
            for i in range(len(patches[j])):
                if i not in context_indices[j]:
                    masked_patches[j][i] = torch.zeros_like(masked_patches[j][i])  # Mask unselected patches to zero

        return masked_patches, context_indices

    def sample_target_blocks(self,patches, m =9,  image_size=(128,128), patch_size=16):
        """
        Randomly selects m continuous patches forming a square block (LxL).

        Args:
            patches (torch.Tensor): Tensor of patches of shape (num_patches, C, patch_size, patch_size).
            image_size (tuple): Original image size as (H, W).
            patch_size (int): Size of each patch (patch_size x patch_size).
            m (int): Number of continuous patches to select.

        Returns:
            list: Indices of selected patches.
            torch.Tensor: Selected patches of shape (m, C, patch_size, patch_size).
        """
        patches = patches.to(device)
        H, W = image_size
        n_patches_h = H // patch_size
        n_patches_w = W // patch_size
        ar = (1.5 - 0.75) * torch.rand((1)) + 0.75 # aspect ratio

        # Compute side length of the target block
        side_length_h = int(ar*(m)**(0.5))
        side_length_w = int(m/side_length_h)

        # Randomly select a top-left corner for the square block
        max_row = n_patches_h - side_length_h
        max_col = n_patches_w - side_length_w
        start_row = random.randint(0, max_row)
        start_col = random.randint(0, max_col)

        # Collect indices of the patches in the square block
        target_indices = []
        for _ in range(patches.shape[0]):
            target_indices.append([
            (start_row + i) * n_patches_w + (start_col + j)
            for i in range(side_length_h)
            for j in range(side_length_w)
        ])

        masked_patches = patches.clone().to(device)
        for j in range(patches.shape[0]):
            for i in range(len(patches[j])):
                if i not in target_indices[j]:
                    masked_patches[j][i] = torch.zeros_like(masked_patches[j][i])  # Mask unselected patches to zero

        return masked_patches, target_indices

    def remove_overlaps(self,context_blocks, context_indices, NUM_TARGETS=4):

        TARGET_INDICES = []
        for i in  range(NUM_TARGETS):
            target,indices = sample_target_blocks(patches)
            target = target.to(device)
            TARGET_INDICES.append(indices)

            if i == 0 :
                TARGET_BLOCKS = target.clone().to(device)
            else:
                TARGET_BLOCKS = torch.cat((TARGET_BLOCKS,target),dim=1).to(device)

            for i in range(len(indices)):
                for idx in indices[i]:
                    context_indices_i = set(context_indices[i])
                    if idx  in context_indices_i:
                        context_blocks[i][idx] = torch.zeros_like(context_blocks[i][idx])
                        context_indices[i].remove(idx)
            context_blocks, TARGET_BLOCKS = context_blocks.to(device), TARGET_BLOCKS.to(device)
        return context_blocks, context_indices ,TARGET_BLOCKS.reshape(-1,4,64,3,16,16), TARGET_INDICES

    def forward(self, x):

        patches = self.divide_into_patches(x)
        context, context_indices = self.sample_context_blocks(patches)
        context, context_indices,targets, target_indices  = self.remove_overlaps(context, context_indices)
        return context, context_indices, targets, target_indices





code taken from - https://github.com/facebookresearch/ijepa

In [ ]:
class GetPositionalEmbeddings(nn.Module):
    def __init__(self):
        super(GetPositionalEmbeddings, self).__init__()

    def get_1d_sincos_pos_embed_from_grid(self,embed_dim, pos):
        """
        embed_dim: output dimension for each position
        pos: a list of positions to be encoded: size (M,)
        out: (M, D)
        """
        assert embed_dim % 2 == 0
        omega = np.arange(embed_dim // 2, dtype=float)
        omega /= embed_dim / 2.
        omega = 1. / 10000**omega   # (D/2,)

        pos = pos.reshape(-1)   # (M,)
        out = np.einsum('m,d->md', pos, omega)   # (M, D/2), outer product

        emb_sin = np.sin(out)  # (M, D/2)
        emb_cos = np.cos(out)  # (M, D/2)

        emb = np.concatenate([emb_sin, emb_cos], axis=1)  # (M, D)
        return emb

    def get_1d_sincos_pos_embed(self,embed_dim, grid_size, cls_token=False):
        """
        grid_size: int of the grid length
        return:
        pos_embed: [grid_size, embed_dim] or [1+grid_size, embed_dim] (w/ or w/o cls_token)
        """
        grid = np.arange(grid_size, dtype=float)
        pos_embed = self.get_1d_sincos_pos_embed_from_grid(embed_dim, grid)
        if cls_token:
            pos_embed = np.concatenate([np.zeros([1, embed_dim]), pos_embed], axis=0)
        return pos_embed


    def get_2d_sincos_pos_embed_from_grid(self,embed_dim, grid):
        assert embed_dim % 2 == 0

        # use half of dimensions to encode grid_h
        emb_h = self.get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[0])  # (H*W, D/2)
        emb_w = self.get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[1])  # (H*W, D/2)

        emb = np.concatenate([emb_h, emb_w], axis=1)  # (H*W, D)
        return emb

    def get_2d_sincos_pos_embed(self,embed_dim, grid_size, cls_token=False):
        """
        grid_size: int of the grid height and width
        return:
        pos_embed: [grid_size*grid_size, embed_dim] or [1+grid_size*grid_size, embed_dim] (w/ or w/o cls_token)
        """
        grid_h = np.arange(grid_size, dtype=float)
        grid_w = np.arange(grid_size, dtype=float)
        grid = np.meshgrid(grid_w, grid_h)  # here w goes first
        grid = np.stack(grid, axis=0)

        grid = grid.reshape([2, 1, grid_size, grid_size])
        pos_embed = self.get_2d_sincos_pos_embed_from_grid(embed_dim, grid)
        if cls_token:
            pos_embed = np.concatenate([np.zeros([1, embed_dim]), pos_embed], axis=0)
        return pos_embed

    def forward(self, grid_size):
        pos_emb = self.get_2d_sincos_pos_embed(768, grid_size)
        return pos_emb

In [ ]:
get_positional_embeddings = GetPositionalEmbeddings().to(device)
patchembedder  = GetPatchEmbeddings().to(device)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_dim: int, num_heads: int):
        super(MultiHeadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads

        self.head_dim = embed_dim // num_heads
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        self.fc_out = nn.Linear(embed_dim, embed_dim)

    def forward(self, query, key, value):
        batch_size = query.shape[0]

        # Linear projections
        Q = self.query(query)  # (batch_size, seq_len, embed_dim)
        K = self.key(key)      # (batch_size, seq_len, embed_dim)
        V = self.value(value)  # (batch_size, seq_len, embed_dim)

        # Split into multiple heads
        Q = Q.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)  # (batch_size, num_heads, seq_len, head_dim)
        K = K.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)  # (batch_size, num_heads, seq_len, head_dim)
        V = V.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)  # (batch_size, num_heads, seq_len, head_dim)

        # Scaled dot-product attention
        energy = torch.matmul(Q, K.transpose(-2, -1)) / (self.head_dim ** 0.5)  # (batch_size, num_heads, seq_len, seq_len)

        attention = torch.softmax(energy, dim=-1).to(device)  # (batch_size, num_heads, seq_len, seq_len)
        out = torch.matmul(attention, V).to(device)  # (batch_size, num_heads, seq_len, head_dim)

        # Concatenate heads and pass through final linear layer
        out = out.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.head_dim)
        out = self.fc_out(out)  # (batch_size, seq_len, embed_dim)

        return out






In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_hid_dim):
        super(TransformerEncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(embed_dim, num_heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_hid_dim),
            nn.ReLU(),
            nn.Linear(ff_hid_dim, embed_dim)
        )


    def forward(self, x ):
        # Attention
        attention_out = self.attention(x, x, x).to(device)
        x = self.norm1(attention_out + x)  # Residual connection

        # Feed-forward network
        ffn_out = self.ffn(x)
        x = self.norm2(ffn_out + x)  # Residual connection

        return self.dropout(x)

In [ ]:
class VisionTransformer(nn.Module):
    def __init__(self,patchembedder,positionembedder,  patch_size, in_channels, embed_dim, num_heads,
                 num_layers, ff_hid_dim, max_len: int = 512, target = False):
        super(VisionTransformer, self).__init__()

        self.embed_dim = embed_dim
        self.patch_size = patch_size
        self.num_patches = (image_size // patch_size) ** 2
        self.patch_embedding = patchembedder
        self.position_embedding = positionembedder
        self.target = target

        # Transformer encoder layers
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(embed_dim, num_heads, ff_hid_dim, dropout)
            for _ in range(num_layers)
        ]).to(device)


    def forward(self, x: torch.Tensor, target_rep: Optional[torch.Tensor] = None, target_indices :Optional [list] = None):
        # Step 1: Patch embedding
        if self.target == False:
            batch_size = x.size(0)
            x,_,target,target_indices = self.patch_embedding(x) # (B, embed_dim, H/patch_size, W/patch_size)

            x = x.flatten(2) # (B, num_patches, embed_dim)

            # Step 2: Add positional embedding
            pos_emb = torch.from_numpy(self.position_embedding(8)).unsqueeze(dim=0).repeat_interleave(repeats = 32, dim=0).to(device)
            pos_emb = pos_emb.float()
            x = x +  pos_emb

            # Step 3: Pass through transformer layers
            for layer in self.encoder_layers:
                x = layer(x)

            # Step 4: Return output for each patch
            x, target  =x.to(device), target.to(device)
            return x , target, target_indices # (B, num_patches, embed_dim)
        else:
            batch_size = x.size(0)
            target_representations = []
            target_blocks = target_rep.to(device)
            for i in range(target_blocks.shape[1]):
                x = target_blocks[:,i,:,:].flatten(2) # (B, num_patches, embed_dim)

                # Step 2: Add positional embedding
                pos_emb = torch.from_numpy(self.position_embedding(8)).unsqueeze(dim=0).repeat_interleave(repeats = 32, dim=0).to(device)
                pos_emb = pos_emb.float()
                x = x+  pos_emb

                # Step 3: Pass through transformer layers
                for layer in self.encoder_layers:
                    x = layer(x)

                target_representations.append(x)
            return target_representations  # (B, num_patches, embed_dim)




In [ ]:
context_enocder = VisionTransformer(
        patchembedder,
        get_positional_embeddings,
        image_size=128,
        patch_size=16,
        in_channels=3,
        embed_dim=768,
        num_heads=8,
        num_layers=6,
        ff_hid_dim=768,
        dropout=0.1,
        target=False
    )
target_encoder = VisionTransformer(
        patchembedder,
        get_positional_embeddings,
        image_size=128,
        patch_size=16,
        in_channels=3,
        embed_dim=768,
        num_heads=8,
        num_layers=6,
        ff_hid_dim=768,
        dropout=0.1,
        target=True
    )

In [ ]:
class VisionTransformer_predictor(nn.Module):
    def __init__(self,positionembedder, patch_size, image_size, in_channels, embed_dim, num_heads,
                 num_layers, ff_hid_dim, max_len: int = 512):
        super(VisionTransformer_predictor, self).__init__()

        self.embed_dim = embed_dim
        self.patch_size = patch_size
        self.num_patches = (image_size // patch_size) ** 2
        self.position_embedding = positionembedder.to(device)

        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(embed_dim, num_heads, ff_hid_dim)
            for _ in range(num_layers)
        ]).to(device)
        self.main = nn.Sequential(
            nn.Linear(1, 64),
            nn.ReLU(),
            nn.Linear(64, 768)
        )
    def get_mask_embeddings(self,target_list : list):

        pos_emb = torch.from_numpy(self.position_embedding(8)).unsqueeze(dim=0).repeat_interleave(repeats = 32, dim=0).to(device)
        pos_emb = pos_emb.float()
        for i in range(len(target_list)):
            pos_masks = self.position_embedding.get_1d_sincos_pos_embed_from_grid(768, pos = np.array(target_list[i]))
            pos_masks = torch.from_numpy(pos_masks).to(device)
            pos_masks = torch.cat([pos_masks, torch.zeros(pos_emb[i].shape[0] - pos_masks.shape[0], 768).to(device)], dim=0).to(device)
            pos_emb[i] =  pos_emb[i] + pos_masks
        return pos_emb

    def forward(self, x: torch.Tensor, target_lists: list):
        x = x.to(device)
        pos_emb = self.get_mask_embeddings(target_lists)
        pos_emb = pos_emb.float()
        x = x + pos_emb

        # Step 3: Pass through transformer layers
        for layer in self.encoder_layers:
            x = layer(x)

        return x






In [ ]:
predictor = VisionTransformer_predictor(
        get_positional_embeddings,
        image_size=128,
        patch_size=16,
        in_channels=3,
        embed_dim=768,
        num_heads=4,
        num_layers=6,
        ff_hid_dim=384,

    )

In [ ]:
def loss_fn(context, target_representaions, target_indices, predictor_model):
    loss = 0
    for i in range(len(target_representaions)):
        context_representations = predictor_model(context.detach(), target_indices[i])
        loss  += torch.mean(torch.sqrt(torch.sum(torch.square(context_representations-target_representaions[i]))))

    return loss/len(target_representaions)

In [ ]:
def train_iJEPA(context_model, target_model, predictor_model, num_epochs = 10):
    context_model  = context_model.to(device)
    target_model = target_model.to(device)
    predictor_model = predictor_model.to(device)
    dataloader = torch.utils.data.DataLoader(animal_dataset, batch_size =32 , shuffle = True, num_workers=True)
    context_opt = torch.optim.Adam(context_model.parameters(), lr = 1e-4)
    predictor_opt = torch.optim.Adam(predictor_model.parameters(), lr  =1e-4)

    for param in target_model.parameters():
        param.requires_grad = False
    train_losses = []
    context_model.train()
    predictor_model.train()
    for epochs in range(num_epochs):
        avg_loss = 0
        for x,_ in tqdm(dataloader):
            with torch.autograd.set_detect_anomaly(True):
                x = x.to(device)
                context_opt.zero_grad()
                predictor_opt.zero_grad()
                context, target, target_indices = context_model(x)
                with torch.no_grad():
                    target_representations = target_model(x,target, target_indices)
                loss = loss_fn(context.detach(), target_representations, target_indices, predictor_model)
                avg_loss += loss.item()

                loss.backward(retain_graph = True, create_graph  =True)
                context_opt.step()
                predictor_opt.step()

                with torch.no_grad():
                    for param_q, param_k in zip(context_model.parameters(), target_model.parameters()):
                        param_k.data.mul_(0.996).add_((1.-0.996) * param_q.detach().data)

        avg_loss/=len(dataloader)
        train_losses.append(avg_loss)
        print(f"epoch:{epochs} | avg_loss: {avg_loss}")


    print('finished Training')